In [1]:
import random
from transformers import pipeline

2025-08-04 12:42:19.782815: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754311340.155595      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754311340.256236      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
try:
    print("Loading sentiment analysis model...")
    sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
    print("Model loaded successfully.\n")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure you have an internet connection to download the model.")
    exit()

Loading sentiment analysis model...


Device set to use cuda:0


Model loaded successfully.



In [7]:
def get_sentiment(text):
    """
    Analyzes the sentiment of a given text using the loaded pipeline.
    Returns the predicted label and score.
    """
    result = sentiment_pipeline(text)[0]
    return result['label'], result['score']


def perform_character_swap_attack(original_text):
    """
    Performs a character swap attack on a random word in the text.
    The goal is to flip the sentiment from POSITIVE to NEGATIVE.
    """
    print("Starting character swap attack...")

    # Get the original sentiment
    original_label, original_score = get_sentiment(original_text)
    print(f"Original Text: '{original_text}'")
    print(f"Original Sentiment: {original_label} (Score: {original_score:.4f})\n")

    # Only proceed if the original sentiment is positive
    if original_label != 'POSITIVE':
        print("Original text is not positive. The attack is designed to flip from POSITIVE to NEGATIVE.")
        return

    words = original_text.split()
    
    # Shuffle words to randomly select a target for the attack
    word_indices = list(range(len(words)))
    random.shuffle(word_indices)

    for i in word_indices:
        word_to_swap = words[i]
        
        # We need a word with at least two characters to swap
        if len(word_to_swap) > 1:
            print(f"Attempting to swap characters in the word: '{word_to_swap}'")
            
            # Select two adjacent characters to swap
            char_indices = list(range(len(word_to_swap) - 1))
            random.shuffle(char_indices)

            for char_pos in char_indices:
                # Create the perturbed word
                word_list = list(word_to_swap)
                word_list[char_pos], word_list[char_pos + 1] = word_list[char_pos + 1], word_list[char_pos]
                perturbed_word = "".join(word_list)

                # Create the new sentence
                perturbed_words = list(words)
                perturbed_words[i] = perturbed_word
                perturbed_text = ' '.join(perturbed_words)

                # Get the sentiment of the perturbed text
                perturbed_label, perturbed_score = get_sentiment(perturbed_text)
                print(perturbed_text)
                print(perturbed_label)
                print(perturbed_score, "\n")
                
                if perturbed_label != original_label:
                    print("-" * 50)
                    print("ATTACK SUCCESSFUL")
                    print(f"Perturbed Text: '{perturbed_text}'")
                    print(f"Flipped Sentiment: {perturbed_label} (Score: {perturbed_score:.4f})")
                    print(f"The word '{word_to_swap}' was changed to '{perturbed_word}'.")
                    print("-" * 50)
                    return

    print("\nAttack failed. Could not find a character swap to flip the sentiment.")

In [8]:
target_text = "The movie was excellent and the plot was engaging."
perform_character_swap_attack(target_text)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Starting character swap attack...
Original Text: 'The movie was excellent and the plot was engaging.'
Original Sentiment: POSITIVE (Score: 0.9999)

Attempting to swap characters in the word: 'and'
The movie was excellent nad the plot was engaging.
POSITIVE
0.9997931122779846 

The movie was excellent adn the plot was engaging.
POSITIVE
0.9997223019599915 

Attempting to swap characters in the word: 'The'
Teh movie was excellent and the plot was engaging.
POSITIVE
0.9998670816421509 

hTe movie was excellent and the plot was engaging.
POSITIVE
0.9998644590377808 

Attempting to swap characters in the word: 'the'
The movie was excellent and teh plot was engaging.
POSITIVE
0.9998564720153809 

The movie was excellent and hte plot was engaging.
POSITIVE
0.9998471736907959 

Attempting to swap characters in the word: 'was'
The movie was excellent and the plot aws engaging.
POSITIVE
0.9998385906219482 

The movie was excellent and the plot wsa engaging.
POSITIVE
0.9998424053192139 

Attempti